In [2]:
import cv2 as cv
import mediapipe as mp
import time

In [3]:
def image_overley(img1,img2,location):
    h,w,_ = img1.shape
    h1,w1 = img2.shape[:2]
    x,y = location
    img1[y:y+h1 , x:x+w1] = img2
    return img1
    

In [4]:
import numpy as np
def image_overlay_sec(img1,img2,location):
    h,w = img2.shape[:2]
    x,y = location
    shapes = np.zeros_like(img1,np.uint8)
    shapes[y:y+h,x:x+w] = img2
    alpha =0.3
    mask = shapes.astype(bool)
    img1[mask] = cv.addWeighted(img1,alpha,shapes,1-alpha,None)[mask]
    return img1

In [5]:
import numpy as np
def image_overlay_third(img1,img2,location):
    h,w = img1.shape[:2]
    h1,w1 = img2.shape[:2]
    x,y = location
    roi = img1[y:y+h1 , x:x+w1]
    
    gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
    _,mask = cv.threshold(gray,150,255,cv.THRESH_BINARY)
    mask_inv = cv.bitwise_not(mask)
    
    img_bg = cv.bitwise_and(roi,roi,mask=mask)
    img_fg = cv.bitwise_and(img2,img2,mask=mask_inv)
    
    final = cv.add(img_bg,img_fg)
    
    img1[y:y+h1,x:x+w1] = final

    return img1

In [6]:
def nothing(x):
    pass

In [7]:
if __name__ == '__main__':

    
    cap = cv.VideoCapture(0)
    
    overlay = cv.imread("./hairstyle.png")
    overlay = cv.resize(overlay ,(155,165))
    # object detection
    cv.namedWindow("bar")

    cv.createTrackbar("u_h","bar",0,255,nothing)
    cv.createTrackbar("u_s","bar",0,255,nothing)
    cv.createTrackbar("u_v","bar",0,255,nothing)

    cv.createTrackbar("l_h","bar",0,255,nothing)
    cv.createTrackbar("l_s","bar",0,255,nothing)
    cv.createTrackbar("l_v","bar",0,255,nothing)
    mpface = mp.solutions.face_detection
    face = mpface.FaceDetection()

    mpdraw = mp.solutions.drawing_utils
    
    while cap.isOpened():
        ret,img = cap.read()
        img = cv.resize(img ,(500,400))
            
        img3 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        results = face.process(img3)
        #print(results.detections)
        


        
        
        frame = cv.cvtColor(img,cv.COLOR_BGR2HSV_FULL)
    

        u_h = cv.getTrackbarPos("u_h","bar")
        u_s = cv.getTrackbarPos("u_s","bar")
        u_v = cv.getTrackbarPos("u_v","bar")

        l_h = cv.getTrackbarPos("l_h","bar")
        l_s= cv.getTrackbarPos("l_s","bar")
        l_v= cv.getTrackbarPos("l_v","bar")

        mask = cv.inRange(frame,(l_h,l_s,l_v),(u_h,u_s,u_v))

        dialation = cv.dilate(mask,(10,10))
        
        retf,thresh = cv.threshold(mask,150,255,cv.THRESH_BINARY)

        contour,her = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

        cv.imshow("thresh",thresh)
    
        
    
        cv.imshow("mask",mask)
        if results.detections:
            for id,detection in enumerate(results.detections):
            #mpdraw.draw_detection(frame,detection)
                bbox = detection.location_data.relative_bounding_box
                ih,iw,ic = img3.shape
                cv.rectangle(img,(int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-40),(int((bbox.xmin + bbox.width)* iw),int((bbox.ymin + bbox.height)*ih)),(255,0,255),2)
                roi = img[int(bbox.ymin *ih)-40:int((bbox.ymin + bbox.height)*ih),int(bbox.xmin *iw)-10:int((bbox.xmin + bbox.width)* iw)]
                after_overlay=image_overlay_third(img,overlay,location = (int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-80))
        for c in contour :
            area = cv.contourArea(c)
            epsilon = 0.00001 * cv.arcLength(c,True)
            data = cv.approxPolyDP(c,epsilon , True)

            hull = cv.convexHull(data)
            if area > 700 :
                #cv.drawContours(after_overlay,[c],-1,(50,50,150),2)
                cv.drawContours(after_overlay,[hull],-1,(0,255,0),2)
    
        
        cv.imshow("img",after_overlay)
        
    
        if cv.waitKey(1) == 27 :
            break
    cap.release()
    cv.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'
